In [ ]:
#!pip install scrapy

In [ ]:
# Importing all of the necessary packages.
import requests
import time
import pandas as pd
import numpy as np
import scrapy
import logging
from scrapy.http import TextResponse
from scrapy.crawler import CrawlerProcess
import re
import os

In [ ]:
base_url = "https://www.menu.am"

In [ ]:
# Scraping the required features using the full scrapy framework. 
# Storing the scraped information in a csv file. 
class MenuAmScraper(scrapy.Spider):
    name = "menu_am"
    start_urls = ["https://www.menu.am/?status=all&sort=default"]
    
    custom_settings = {
        "LOG_LEVEL": logging.WARNING,
        "FEED_FORMAT": "csv",
        "FEED_URI": "Menu_am.csv"
    }
    
    def parse(self, response):
        names = []
        opening_hrs = []
        ratings_partial = []
        ratings = []
        categories = []
        hyperlinks = []

        names = response.css("div.fl.list-title>a.title::text").extract()
        opening_hrs = response.css("div.new_list_time_block>span.new_list_time_block_inner::text").extract()
        rating_blocks = response.xpath("//div[contains(@class, 'item ')]")
        categories = response.css("span.restType::text").extract()
        restaurant_url = response.css("div.fl.list-title>a::attr(href)").extract()
        hyperlinks = [base_url + i for i in restaurant_url]
  

        for a in rating_blocks:
            ratings_partial.append(a.xpath('div[@class="new_favorites_and_rates_block"]//text()').extract())
        for b in ratings_partial:
            if(b == []):
                ratings.append(0)
                continue
            ratings.append(b[0].strip())
        for names_csv, opening_hrs_csv, ratings_csv, categories_csv, hyperlinks_csv in zip(names, opening_hrs, ratings, categories, hyperlinks):
            yield {'Names': names_csv, 'Opening_Hours': opening_hrs_csv, 'Ratings': ratings_csv, 'Categories': categories_csv, 'Hyperlinks': hyperlinks_csv}


if(os.path.isfile('Menu_am.csv') == False):
    process = CrawlerProcess()
    process.crawl(MenuAmScraper)
    process.start()


In [ ]:
# Creating a DataFrame using our previously created csv file
# in order to answer the following questions.
df = pd.read_csv('Menu_am.csv')

In [ ]:
# Question 1. Grouping Ratings by Categories, calculating the mean, and using
# sort_values() function in order to see on average which Categories have the 
# highest rating. Below you can find 5 categories. 
df['Ratings'].groupby(df['Categories']).mean().sort_values(ascending = False).head(5)

Categories
Արաբական                                  5.0
Սուրճ և Թեյ, Ֆրանսիական                   5.0
Լավ առաջարկ                               5.0
Գեղեցկություն և առողջություն, Մանկական    5.0
Ծաղիկներ և Փուչիկներ, Քաղցրավենիք         5.0
Name: Ratings, dtype: float64

In [ ]:
# Question 2. Creating 2 lists. In the 1st list we're storing our data from Opening_Hours 
# column. The 2nd one is empty, where we'll store our time equal to 7pm or sooner. 
# With the help of for loop we're trying to find our needed time and append it to
# our empty list. Last but not least, in order to see the results we can use the .loc for accessing 
# group of values using labels, and .isin for an equivalent method on series. 
time = df['Opening_Hours']
seven_or_sooner = []
for i in time:
    if(i[8:10] != ''):
        if(int(i[8:10])<=19 and int(i[8:10])>12):
            seven_or_sooner.append(i)
df['Names'].loc[df['Opening_Hours'].isin(seven_or_sooner)]

12                                Լեդի Ջի Քեյք 
16                                    Էլեն Քեյք
23                          Տաք Կճուճ Ֆուդ Կորտ
24                                 ՍԻՍ Նատուրալ
29                       Ղարաբաղի Ժենգյալով Հաց
31                                    Թեյք Ուեյ
33                                  Ռեդի Լանչիս
34                                      Պատառիկ
38                                 Խմորի Աշխարհ
45                      Գևորգյան Գինու Գործարան
47                            Անտարես Գրախանութ
54                               Ֆուրշետ Բուֆետ
56                                         Չոկո
60                                       Բոնիտո
62                                  Ապարան Ջուր
95                                     Գրին Դեյ
99                                       ԲիԷլլա
110                                  Փարթի Թայմ
112                                    Արտ Լանչ
123                                  Դան Դեսերտ
128                                Բագետ